In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Seqbased_model/RF_1302_models \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023


***
### Load Data :

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter, defaultdict
import warnings
import logging
import subprocess
from multiprocessing.pool import ThreadPool
import joblib

# SCikitlearn modules :
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report , roc_auc_score, matthews_corrcoef
from sklearn.metrics import roc_auc_score

# Scipy modules : 
from scipy.stats import fisher_exact
#from skopt import BayesSearchCV
#from skopt.space import Real, Categorical, Integer
from statistics import mean

path_work = "/media/concha-eloko/Linux/PPT_clean"
path_jobs = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models"

> Set the count prophage dico : 

In [2]:
DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)

DF_info = DF_info.drop_duplicates(subset = ["Protein_name"])

df_prophages = DF_info.drop_duplicates(subset = ["Phage"], keep = "first")
dico_prophage_info = {row["Phage"] : {"prophage_strain" : row["prophage_id"] , "ancestor" : row["Infected_ancestor"]} for _,row in df_prophages.iterrows()}

def get_filtered_prophages(prophage) :
    combinations = []
    to_exclude = set()
    to_keep = set()
    to_keep.add(prophage)
    df_prophage_group = DF_info[(DF_info["prophage_id"] == dico_prophage_info[prophage]["prophage_strain"]) & (DF_info["Infected_ancestor"] == dico_prophage_info[prophage]["ancestor"])]
    if len(df_prophage_group) == 1 :
        pass
    else :
        depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage]["domain_seq"].values)
        for prophage_tmp in df_prophage_group["Phage"].unique().tolist() :
            if prophage_tmp != prophage :
                tmp_depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage_tmp]["domain_seq"].values)
                if depo_set == tmp_depo_set :
                    to_exclude.add(prophage_tmp)
                else :
                    if tmp_depo_set not in combinations :
                        to_keep.add(prophage_tmp)
                        combinations.append(tmp_depo_set)
                    else :
                        to_exclude.add(prophage_tmp)
    return df_prophage_group , to_exclude , to_keep

good_prophages = set()
excluded_prophages = set()

for prophage, info_prophage in tqdm(dico_prophage_info.items()) :
    if prophage not in excluded_prophages and prophage not in good_prophages:
        _, excluded_members , kept_members = get_filtered_prophages(prophage)
        good_prophages.update(kept_members)
        excluded_prophages.update(excluded_members)

DF_info_lvl_0_filtered = DF_info[DF_info["Phage"].isin(good_prophages)]
DF_info_lvl_0_final = DF_info_lvl_0_filtered[~DF_info_lvl_0_filtered["KL_type_LCA"].str.contains("\\|")]
DF_info_lvl_0 = DF_info_lvl_0_final.copy()

df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15981/15981 [00:22<00:00, 718.88it/s]


In [13]:
cluster_values = set([file.split("_")[0] for file in os.listdir(path_jobs)])
KLtypes = set([file.split("_RF_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_RF_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes}

KLtypes_paths

{'KL71': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.9_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.75_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.8_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.7_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.975_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.95_RF_KL71.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.65_RF_KL71.full_data.joblib'],
 'KL139': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.75_RF_KL139.full_data.joblib',
  '/media/concha-eloko/Linux/PPT_clean/ficheros

***
### Get the best cl value :

> Functions :

In [6]:
def get_mean_MCC(path_data_object):
    try:
        assert os.path.isfile(path_data_object)
        data_object = joblib.load(path_data_object)
        mcc_values = []
        for i in data_object:
            y_test = data_object[i]["test_data"][0].values
            predictions = data_object[i]["test_data"][1]
            mcc = matthews_corrcoef(y_test, predictions)
            mcc_values.append(mcc)
        del data_object 
        mean_mcc = mean(mcc_values)
        return mean_mcc
    except AssertionError:
        raise FileNotFoundError(f"File not found: {path_data_object}")

    

> Get the best model : 

In [14]:
%%time
with open(f"/media/concha-eloko/Linux/PPT_clean/Seqbased_model/Seqbased_models.prophages_metrics.tsv", "w") as outfile :
    for KLtype in tqdm(KLtypes_paths) : 
        for path in KLtypes_paths[KLtype] :
            cl_value = path.split("/")[-1].split("_RF")[0]
            mean_mcc = get_mean_MCC(path)
            count_kltype = dico_prophage_count[KLtype]
            outfile.write(f"{KLtype}\t{count_kltype}\t{cl_value}\t{mean_mcc}\n")
            print(f"{KLtype}\t{count_kltype}\t{cl_value}\t{mean_mcc}")


  0%|                                                                                                                                     | 0/96 [00:00<?, ?it/s]

KL71	15	0.9	0.5423261445466404
KL71	15	0.75	0.7409207609430039
KL71	15	0.85	0.0
KL71	15	0.8	0.7409207609430039
KL71	15	0.7	0.5919747986457313
KL71	15	0.975	0.4338609156373123
KL71	15	0.95	0.4338609156373123


  1%|█▎                                                                                                                           | 1/96 [00:07<11:41,  7.38s/it]

KL71	15	0.65	0.5015354472338103
KL139	10	0.75	0.7164117158323701
KL139	10	0.9	0.735400640077266
KL139	10	0.85	0.6994347603915498
KL139	10	0.7	0.6911004289095706
KL139	10	0.65	0.6911004289095706
KL139	10	0.8	0.8416176409860249
KL139	10	0.975	0.0


  2%|██▌                                                                                                                          | 2/96 [00:12<09:37,  6.14s/it]

...

KL149	70	0.65	0.8642699761248377
KL152	10	0.8	0.4148399724926484
KL152	10	0.65	0.4296799449852968
KL152	10	0.85	0.4696799449852968
KL152	10	0.975	0.4696799449852968
KL152	10	0.95	0.4696799449852968
KL152	10	0.75	0.4696799449852968
KL152	10	0.7	0.4148399724926484


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 95/96 [11:06<00:07,  7.14s/it]

KL152	10	0.9	0.4696799449852968
KL117	17	0.9	0.584665064908091
KL117	17	0.975	0.5526986749914522
KL117	17	0.95	0.584665064908091
KL117	17	0.8	0.6612231023961064
KL117	17	0.65	0.3694527909108401
KL117	17	0.75	0.728442672656688
KL117	17	0.85	0.48081243640493787


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [11:12<00:00,  7.00s/it]

KL117	17	0.7	0.7779173459253124


In [15]:
threshold = 15

names_metric_col = ["KL_type", "Count", "c_value","mean_mcc"]
metric_df = pd.read_csv(f"/media/concha-eloko/Linux/PPT_clean/Seqbased_model/Seqbased_models.prophages_metrics.tsv", sep = "\t", names = names_metric_col)

metric_eval_df = metric_df[metric_df["Count"] > threshold]

weighted_mcc_dico = {}
for cluster in metric_eval_df["c_value"].unique().tolist() :
    mcc_sum = 0
    cl_df = metric_eval_df[metric_eval_df["c_value"] == cluster]
    for _,row in cl_df.iterrows() : 
        mcc_sum = mcc_sum + row["mean_mcc"] * row["Count"]
    weighted_mcc = mcc_sum / (sum(cl_df["Count"]))
    weighted_mcc_dico[cluster] = weighted_mcc

weighted_mcc_dico

{0.9: 0.7751099285229077,
 0.975: 0.741274524549705,
 0.8: 0.7697635340143436,
 0.65: 0.7708792611301989,
 0.95: 0.7567348331288974,
 0.75: 0.761438770189088,
 0.7: 0.7621235953812278,
 0.85: 0.7772843284953614}

***
### Get the full metrics : 

> Relevant objects path :

In [16]:
cl_value = "0.85"

KLtypes = set([file.split("_RF_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_")[0] == cl_value if file.split("_RF_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes if dico_prophage_count[kltype] > threshold}



In [17]:
KLtypes_paths

{'KL15': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL15.full_data.joblib'],
 'KL145': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL145.full_data.joblib'],
 'KL110': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL110.full_data.joblib'],
 'KL125': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL125.full_data.joblib'],
 'KL20': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL20.full_data.joblib'],
 'KL107': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL107.full_data.joblib'],
 'KL128': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL128.full_data.joblib'],
 'KL38': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_RF_KL38.full_data.joblib'],
 'KL45': ['/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/RF_1302_models/0.85_R

In [11]:
def get_mean_MCC(path_data_object) :
    data_object = joblib.load(f"{path_data_object}")
    mcc_values = []
    for i in data_object :
        y_test = data_object[i]["test_data"][0].values
        predictions = data_object[i]["test_data"][1]
        mcc = matthews_corrcoef(y_test, predictions)
        mcc_values.append(mcc)
    del data_object
    mean_mcc = mean(mcc_values)
    return mean_mcc

def get_mean_metrics(path_data_object) :
    data_object = joblib.load(f"{path_data_object}")
    mcc_values = []
    auc_values = []
    recall_values = []
    f1_values = []
    acc_values = []
    prec_values = []
    report_list = []
    for i in data_object :
        y_test = data_object[i]["test_data"][0].values
        predictions = data_object[i]["test_data"][1]
        mcc = matthews_corrcoef(y_test, predictions)
        auc = roc_auc_score(y_test, predictions)
        auc_values.append(auc)
        mcc_values.append(mcc)
        # Report : 
        report = classification_report(y_test, predictions, output_dict=True)
        recall_values.append(report["1"]["recall"])
        f1_values.append(report["1"]["f1-score"])
        prec_values.append(report["1"]["precision"])
        acc_values.append(report["accuracy"])
        report_list.append(report)
    del data_object
    mean_mcc = mean(mcc_values)
    mean_auc = mean(auc_values)
    mean_recall = mean(recall_values)
    mean_f1 = mean(f1_values)
    mean_acc = mean(acc_values)
    mean_prec = mean(prec_values)
    return mean_mcc , mean_recall , mean_f1 , mean_acc , mean_prec , mean_auc



In [18]:
mean_mcc , mean_recall , mean_f1 , mean_acc , mean_prec , mean_auc = get_mean_metrics(KLtypes_paths["KL47"][0])

In [19]:
with open(f"/media/concha-eloko/Linux/PPT_clean/Seqbased_model/Seqbased_models.prophages_FULL_metrics.tsv", "w") as outfile :
    outfile.write(f"KL_type\tn_prophages\tF1\trecall\tprecision\taccuracy\tAUC\tMCC\n")
    for kltype in KLtypes_paths :
        count_kltype = dico_prophage_count[kltype]
        mean_mcc , mean_recall , mean_f1 , mean_acc , mean_prec , mean_auc = get_mean_metrics(KLtypes_paths[kltype][0])
        outfile.write(f"{kltype}\t{count_kltype}\t{mean_f1}\t{mean_recall}\t{mean_prec}\t{mean_acc}\t{mean_auc}\t{mean_mcc}\n")
        print(f"{kltype}\t{count_kltype}\t{mean_f1}\t{mean_recall}\t{mean_prec}\t{mean_acc}\t{mean_auc}\t{mean_mcc}")

KL15	204	0.8114588070327643	0.757026713124274	0.8778850278850279	0.9417461146663184	0.8678337449116517	0.781243516812687
KL145	29	0.9418181818181818	0.9	1.0	0.9833333333333333	0.95	0.9377191850351939
KL110	64	0.7280331262939959	0.6615384615384615	0.8526050420168068	0.9205128205128206	0.816923076923077	0.701556087107071


/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

KL125	26	0.0	0.0	0.0	0.8334677419354839	0.5	0.0


/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

KL20	18	0.0	0.0	0.0	0.8337662337662338	0.5	0.0
KL107	1121	0.8266579480598523	0.8591785714285715	0.7972308587597026	0.9399890351240057	0.9076641520244462	0.79153889872206
KL128	22	0.808008658008658	0.72	0.9666666666666667	0.947008547008547	0.8554545454545455	0.8008969483663161
KL38	76	0.6828671328671329	0.5283333333333333	1.0	0.9211180124223602	0.7641666666666667	0.6911254303953135
KL45	63	0.8955204216073781	0.8282051282051281	0.9833333333333333	0.9683157894736842	0.9125152625152625	0.8840078210857842
KL8	20	0.4780952380952381	0.35	0.9	0.8833333333333333	0.67	0.5027182824148568
KL51	144	0.8402082793189871	0.84064039408867	0.8407739934436264	0.946773759913967	0.9043479748221128	0.8086417598404179
KL5	28	0.6435497835497835	0.5466666666666666	0.9266666666666666	0.913781512605042	0.7664367816091954	0.649713808922839
KL29	37	0.76	0.6678571428571428	0.9047619047619048	0.9367676767676768	0.828523166023166	0.7414910730164936
KL1	190	0.5279575510104982	0.37894736842105264	0.9261403508771929	0.89

...

/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

KL31	16	0.5314285714285715	0.48333333333333334	0.6	0.9057894736842105	0.7354166666666666	0.5060448024364359
KL151	61	0.8819875776397516	0.8038461538461539	0.9818181818181818	0.9644946316179193	0.9002837326607818	0.868651038658241
KL123	29	0.6266666666666667	0.5	0.9333333333333333	0.9055555555555556	0.7433333333333333	0.6298843133197235


/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

KL109	16	0.08	0.05	0.2	0.8436842105263157	0.525	0.0917662935482247
KL2	351	0.834817120753062	0.7534406438631791	0.9374125780431639	0.9504248628395557	0.8716211717899465	0.8131165601721763
KL106	487	0.7956232917703557	0.8200294550810014	0.772739221599344	0.9297887680169311	0.8858838481948975	0.7538109845802491
KL122	29	0.5504761904761905	0.39999999999999997	1.0	0.9	0.7	0.585477207921569
KL70	30	0.8151515151515152	0.7333333333333334	0.9666666666666667	0.95	0.8633333333333334	0.8092852470120718
KL43	44	0.6373809523809524	0.5277777777777778	0.8361904761904762	0.9055152394775037	0.7547979797979798	0.6137062115991918
KL127	33	0.6707070707070707	0.5285714285714286	1.0	0.9196153846153846	0.7642857142857142	0.6853596221955532
KL25	269	0.8380233762041729	0.8104122990915443	0.8712717446785647	0.9479491567794166	0.8929384915532071	0.8090144660938176
KL18	22	0.9555555555555556	0.96	0.96	0.9849002849002849	0.9754545454545455	0.9497259894483203
KL62	152	0.8630735741428277	0.8094623655913978	0.9292857

/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/concha-eloko/Linux/conda_envs/ML_work_v2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

***
### Dump the best model for each KL types :

In [20]:
path_models = "/media/concha-eloko/Linux/PPT_clean/Seqbased_model/1702_models"
cl_value = "0.85"

KLtypes = set([file.split("_RF_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_")[0] == cl_value if file.split("_RF_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes}


In [21]:
def save_best_model(path_data_object) :
    data_object = joblib.load(f"{path_data_object}")
    mcc_values = []
    kltype = path_data_object.split("/")[-1].split("RF_")[1].split(".full")[0]
    for i in range(0,5) :
        y_test = data_object[i]["test_data"][0].values
        predictions = data_object[i]["test_data"][1]
        mcc = matthews_corrcoef(y_test, predictions)
        mcc_values.append(mcc)
    #print(kltype)
    #print(mcc_values)
    #print(np.argmax(mcc_values))
    #print(data_object[np.argmax(mcc_values)])
    #print("\n")
    best_model = data_object[np.argmax(mcc_values)]["model"]
    joblib.dump(best_model, f'{path_models}/{cl_value}_RF_{kltype}.best_RF_model.joblib')
    del data_object
    print(f"Model saved for {kltype}")

for kltype in KLtypes_paths :
    save_best_model(KLtypes_paths[kltype][0])
    


Model saved for KL71
Model saved for KL139
Model saved for KL124
Model saved for KL15
Model saved for KL157
Model saved for KL145
Model saved for KL110
Model saved for KL125
Model saved for KL20
Model saved for KL107
Model saved for KL128
Model saved for KL38
Model saved for KL45
Model saved for KL8
Model saved for KL51
Model saved for KL5
Model saved for KL137
Model saved for KL103
Model saved for KL29
Model saved for KL6
Model saved for KL61
Model saved for KL1
Model saved for KL136
Model saved for KL53
Model saved for KL17
Model saved for KL48
Model saved for KL105
Model saved for KL112
Model saved for KL16
Model saved for KL28
Model saved for KL114
Model saved for KL35
Model saved for KL169
Model saved for KL24
Model saved for KL4
Model saved for KL13
Model saved for KL7
Model saved for KL30
Model saved for KL118
Model saved for KL23
Model saved for KL41
Model saved for KL57
Model saved for KL26
Model saved for KL3
Model saved for KL63
Model saved for KL116
Model saved for KL39
Mod

IndexError: list index out of range